In [1]:
#import nbimporter
import pickle
import zipfile
import datetime
import string
import glob
import math
import os
import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.model_selection
import traceback
import kagglehub
#assert tf.test.is_gpu_available()
import random
import string
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
import PIL
import pickle
from PIL import Image,ImageOps
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
import gc
from PIL import Image,ImageOps

In [2]:
from tensorflow.keras.utils import to_categorical

# Cargar dataset

In [3]:
char_coordinates_images_path = kagglehub.dataset_download('gabrielguz/char-coordinates-images')

In [4]:
def load_dataset(dataset_name):
    # Define the paths to the images and labels
    images_dir = os.path.join(dataset_name, 'images')
    labels_path = os.path.join(dataset_name, 'labels.pkl')

    # Load the labels from the pickle file
    with open(labels_path, 'rb') as f:
        labels = pickle.load(f)

    # Get a sorted list of image filenames
    image_files = sorted(os.listdir(images_dir), key=lambda x: int(x.split('_')[1].split('.')[0]))

    # Load the images
    images = []
    for image_file in image_files:
        image_path = os.path.join(images_dir, image_file)
        image = Image.open(image_path)
        images.append(image)

    return images, labels

In [5]:
char_coordinates_images_path='/kaggle/input/char-coordinates-images/content/dataset_abc'
data, labels = load_dataset(char_coordinates_images_path)
print(f'Number of images: {len(data)}')
print(f'Number of labels: {len(labels)}')

Number of images: 30873
Number of labels: 30873


In [6]:

num_samples = len(data)
indices = np.arange(num_samples)
np.random.shuffle(indices)

# Randomizo
data = [data[i] for i in indices]
labels = [labels[i] for i in indices]

In [7]:
num_samples = len(data)
data = data[0:len(data)//2]
labels= labels[ 0:len(labels)//2]

In [8]:
num_samples = len(data)
train_size = int(0.7 * num_samples)
val_size = int(0.2 * num_samples)
test_size = num_samples - train_size - val_size  

In [41]:
image_sizes=(data[0].size)

In [42]:
print(image_sizes)

(244, 244)


In [9]:
# Conjunto de entrenamiento
data_train = data[:train_size]
labels_train = labels[:train_size]

# Conjunto de validación
data_val = data[train_size:train_size + val_size]
labels_val = labels[train_size:train_size + val_size]

# Conjunto de prueba
data_test = data[train_size + val_size:]
labels_test = labels[train_size + val_size:]

In [10]:
print(f"Número de tamaño de entrenamiento: {train_size}\n")
print(f"Número de tamaño de validación: {val_size}\n")
print(f"Número de tamaño de prueba: {test_size}\n")
print(f"Número de imágenes de entrenamiento: {len(data_train)}\n")
print(f"Número de imágenes de validación: {len(data_val)}\n")
print(f"Número de imágenes de prueba: {len(data_test)}\n")

Número de tamaño de entrenamiento: 10805

Número de tamaño de validación: 3087

Número de tamaño de prueba: 1544

Número de imágenes de entrenamiento: 10805

Número de imágenes de validación: 3087

Número de imágenes de prueba: 1544



In [11]:
def verificar_data_no_nulo(a, b, c,d,e,f):
    if a == 0 or b == 0 or c == 0 or (a!=d) or (b!=e) or (c!=f):
        raise ValueError("Error: Una o más variables son cero")

In [12]:
verificar_data_no_nulo(len(data_test),len(data_train),len(data_val),len(labels_test),len(labels_train),len(labels_val))

In [13]:

train_image_dir = 'train_images'
val_image_dir = 'val_images'
test_image_dir = 'test_images'

os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(test_image_dir, exist_ok=True)

In [14]:
# Guardar imágenes y obtener rutas
def save_images(data_subset, image_dir):
    image_paths = []
    for idx, img in enumerate(data_subset):
        image_path = os.path.join(image_dir, f'image_{idx}.png')
        img.save(image_path)
        image_paths.append(image_path)
    return image_paths

train_image_paths = save_images(data_train, train_image_dir)
val_image_paths = save_images(data_val, val_image_dir)
test_image_paths = save_images(data_test, test_image_dir)


In [15]:
#Nuevo
# Paso 5: Preparar las etiquetas
labels_train_class = [label[0] for label in labels_train]
labels_val_class = [label[0] for label in labels_val]
labels_test_class = [label[0] for label in labels_test]

# Extraer las coordenadas
labels_train_coords = [label[1] for label in labels_train]
labels_val_coords = [label[1] for label in labels_val]
labels_test_coords = [label[1] for label in labels_test]

In [16]:
label_encoder = LabelEncoder()
labels_all_classes = labels_train_class + labels_val_class + labels_test_class
labels_encoded = label_encoder.fit_transform(labels_all_classes)
num_classes = len(label_encoder.classes_)

# Dividir las etiquetas codificadas
labels_train_encoded = labels_encoded[:train_size]
labels_val_encoded = labels_encoded[train_size:train_size + val_size]
labels_test_encoded = labels_encoded[train_size + val_size:]

# Convertir a one-hot encoding
labels_train_one_hot = to_categorical(labels_train_encoded, num_classes=num_classes)
labels_val_one_hot = to_categorical(labels_val_encoded, num_classes=num_classes)
labels_test_one_hot = to_categorical(labels_test_encoded, num_classes=num_classes)

In [17]:
print(f"Número de imágenes de entrenamiento: {len(data_train)}")
print(f"Número de etiquetas de clase de entrenamiento: {len(labels_train_one_hot)}")
print(f"Número de coordenadas de entrenamiento: {len(labels_train_coords)}")

Número de imágenes de entrenamiento: 10805
Número de etiquetas de clase de entrenamiento: 10805
Número de coordenadas de entrenamiento: 10805


In [18]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [19]:
def normalize_coordinates(coords, original_width, original_height):
    coords = np.array(coords, dtype='float32')
    x_coords = coords[::2]  # Coordenadas x
    y_coords = coords[1::2]  # Coordenadas y
    x_coords /= original_width
    y_coords /= original_height
    normalized_coords = np.concatenate([x_coords, y_coords], axis=-1)
    return normalized_coords

In [20]:

original_image = Image.open(train_image_paths[0])
original_width, original_height = original_image.size


labels_train_coords_normalized = [normalize_coordinates(coords, original_width, original_height) for coords in labels_train_coords]
labels_val_coords_normalized = [normalize_coordinates(coords, original_width, original_height) for coords in labels_val_coords]
labels_test_coords_normalized = [normalize_coordinates(coords, original_width, original_height) for coords in labels_test_coords]

In [21]:
batch_size=64

In [44]:
def create_dataset(image_paths, labels_one_hot, coords_normalized, shuffle=False):
    image_paths_tensor = tf.constant(image_paths)
    labels_tensor = tf.constant(labels_one_hot)
    coords_tensor = tf.constant(coords_normalized)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths_tensor, labels_tensor, coords_tensor))
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


def load_and_preprocess_image(image_path, label_class, coords):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)

    image = tf.image.resize(image, [224, 224])
    image = preprocess_input(image)

    label_class = tf.cast(label_class, tf.float32)
    coords = tf.cast(coords, tf.float32)
    return image, {'class_output': label_class, 'bbox_output': coords}


In [45]:
train_dataset = create_dataset(train_image_paths, labels_train_one_hot, labels_train_coords_normalized, shuffle=True)
val_dataset = create_dataset(val_image_paths, labels_val_one_hot, labels_val_coords_normalized)
test_dataset = create_dataset(test_image_paths, labels_test_one_hot, labels_test_coords_normalized)

# DEFINO EL MODELO

In [24]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import numpy as np

# Cargar VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [25]:
from tensorflow.keras.layers import GlobalAveragePooling2D

In [26]:
for layer in base_model.layers:
    layer.trainable = False # Congelar las capas


base_output=base_model.output


# Rama de localización
x_loc = GlobalAveragePooling2D()(base_output)
x_loc = layers.Dense(512)(x_loc)
x_loc = layers.Activation('relu')(x_loc)
x_loc = layers.BatchNormalization()(x_loc)
x_loc = layers.Dropout(0.2)(x_loc)

x_loc = layers.Dense(256)(x_loc)
x_loc = layers.BatchNormalization()(x_loc)
x_loc = layers.Activation('relu')(x_loc)
x_loc = layers.Dropout(0.2)(x_loc)

x_loc = layers.Dense(128)(x_loc)
x_loc = layers.BatchNormalization()(x_loc)
x_loc = layers.Activation('relu')(x_loc)

bbox_output = layers.Dense(8, activation='sigmoid', name='bbox_output')(x_loc)

# Rama de clasificación
x_class = GlobalAveragePooling2D()(base_output)
x_class = layers.Dense(512)(x_class)
x_class = layers.Activation('relu')(x_class)
x_class = layers.BatchNormalization()(x_class)
x_class = layers.Dropout(0.5)(x_class)

x_class = layers.Dense(256)(x_class)
x_class = layers.Activation('relu')(x_class)
x_class = layers.BatchNormalization()(x_class)
x_class = layers.Dropout(0.3)(x_class)

x_class = layers.Dense(128)(x_class)
x_class = layers.BatchNormalization()(x_class)
x_class = layers.Activation('relu')(x_class)
x_class = layers.Dropout(0.2)(x_class)

class_output = layers.Dense(num_classes, activation='softmax', name='class_output')(x_class)

# Crear el modelo
model = models.Model(inputs=base_model.input, outputs=[bbox_output, class_output])

ENTRENO PARA LOCALIZACIÓN
Ignoro class_output

In [28]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
for layer in model.layers:
    if 'class' in layer.name:
        layer.trainable = False

# Compilar el modelo para localización
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={'bbox_output': 'mean_squared_error', 'class_output': 'categorical_crossentropy'},
    loss_weights={'bbox_output': 1.0, 'class_output': 0.0},
    metrics={'bbox_output': 'mae'}
)

In [ ]:
for layer in model.layers:
    if 'class' in layer.name:
        layer.trainable = False

# Compilar el modelo para localización
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={'bbox_output': 'mean_squared_error', 'class_output': 'categorical_crossentropy'},
    loss_weights={'bbox_output': 1.0, 'class_output': 0.0},
    metrics={'bbox_output': 'mae'}
)

# Entrenar el modelo
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_loc = model.fit(
    train_dataset,
    epochs=20,
    validation_data=val_dataset,
    callbacks=[early_stopping]
)

ENTRENO PARA CLASIFICACIÓN
Ignoro bbox_output

In [ ]:
for layer in model.layers:
    if 'bbox' in layer.name:
        layer.trainable = False
    elif 'class' in layer.name:
        layer.trainable = True

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={'bbox_output': 'mean_squared_error', 'class_output': 'categorical_crossentropy'},
    loss_weights={'bbox_output': 0.0, 'class_output': 1.0},
    metrics={'class_output': 'accuracy'}
)
history_class = model.fit(
    train_dataset,
    epochs=20,
    validation_data=val_dataset,
    callbacks=[early_stopping]
)


FINETUNEO

In [ ]:
for layer in base_model.layers[-4:]:
    layer.trainable = True

COMPILACIÓN PREVIA

In [ ]:
from tensorflow.keras.optimizers import Adam

# Usar una tasa de aprendizaje baja para el fine-tuning
learning_rate = 1e-5
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss={
        'class_output': 'categorical_crossentropy',
        'bbox_output': 'mean_squared_error'
    },
    loss_weights={
        'class_output': 2.0,
        'bbox_output': 1.0
    },
    metrics={
        'class_output': 'accuracy',
        'bbox_output': 'mae'
    }
)


In [ ]:
history_finetune = model.fit(
    train_dataset,
    epochs=20,
    validation_data=val_dataset,
    callbacks=[early_stopping]
)

# Paso 4: Evaluar el modelo
bbox_output_mae, loss, class_output_accuracy= model.evaluate(test_dataset)
print(f"Loss total en prueba: {loss}")
print(f" Accuracy: {class_output_accuracy}")
print(f" MAE: {bbox_output_mae}")

In [ ]:

def reverse_preprocess_vgg16(image, original_size):
    """
    Revertir el preprocesamiento aplicado por VGG16.
    
    Args:
        image (numpy.ndarray): Imagen preprocesada con VGG16 (BGR, medios restados).
        original_size: Altura original de la imagen y Ancho original de la imagen.
    
    Returns:
        numpy.ndarray: Imagen en formato RGB, valores en [0, 255], redimensionada al tamaño original.
    """
    # Valores medios de ImageNet para VGG16
    MEAN_VALUES = np.array([103.939, 116.779, 123.68])
    
    img = image.copy()
    img += MEAN_VALUES
    img = img[..., ::-1]
    img = np.clip(img, 0, 255).astype('uint8')
    
    # Redimensiona a las dimensiones originales
    img = cv2.resize(img, original_size, interpolation=cv2.INTER_LINEAR)
    
    return img

In [ ]:


def visualize_test_predictions(model,test_dataset,num_images_to_process = 10)
    images_processed = 0
    
    for images, labels in test_dataset:
        preds = model.predict(images)
        preds_coords = preds[0]
        preds_classes = preds[1]
    
        for i in range(len(images)):
            if images_processed >= num_images_to_process:
                break  # Salir del bucle si ya procesamos 5 imágenes
        
            img = images[i].numpy()
            img = reverse_preprocess_vgg16(img, image_sizes)
        
            coords_true = labels['bbox_output'][i].numpy()
            coords_pred = preds_coords[i]
        
            coords_true_scaled = coords_true.copy()
            coords_true_scaled[:4] *= original_width  # x_coords
            coords_true_scaled[4:] *= original_height  # y_coords

            coords_pred_scaled = coords_pred.copy()
            coords_pred_scaled[:4] *= original_width  # x_coords
            coords_pred_scaled[4:] *= original_height  # y_coords

            x_true = coords_true_scaled[:4]
            y_true = coords_true_scaled[4:]
            x_pred = coords_pred_scaled[:4]
            y_pred = coords_pred_scaled[4:]
        
            class_true_idx = np.argmax(labels['class_output'][i].numpy())
            class_pred_idx = np.argmax(preds_classes[i])
            class_true_label = label_encoder.inverse_transform([class_true_idx])[0]
            class_pred_label = label_encoder.inverse_transform([class_pred_idx])[0]
        
            fig, ax = plt.subplots(1)
            ax.imshow(img)
        
            ax.plot(x_true, y_true, 'go-', label='Verdadero')
        
            ax.plot(x_pred, y_pred, 'ro-', label='Predicho')
        
            plt.title(f"Clase real: {class_true_label}, Clase predicha: {class_pred_label}")
            plt.legend()
            plt.show()
        
            images_processed += 1
    
        if images_processed >= num_images_to_process:
            break  # Salir del bucle externo si ya procesamos 5 imágenes

In [ ]:
visualize_test_predictions(model,test_dataset)

In [ ]:
predictions = model.predict(test_dataset)
preds_classes = predictions[1]

In [ ]:
y_true = []


for images, labels in test_dataset:
    class_labels = labels['class_output'].numpy()
    class_indices = np.argmax(class_labels, axis=1)
    y_true.extend(class_indices)

y_true = np.array(y_true)

In [ ]:
y_pred = np.argmax(preds_classes, axis=1)

In [ ]:
from sklearn.metrics import classification_report

class_labels = label_encoder.classes_
report = classification_report(y_true, y_pred, target_names=class_labels)

print(report)